# Backtesting Script

### Should be careful not to test on tickers & timeline from training set

In [1]:
import pandas as pd
import numpy as np
import time, datetime
import math

In [3]:
import pandas_datareader as web

In [4]:
tickers = ['AMD', 'HEXO', 'SAVE', 'OGI', 'MOGU', 'CRVS', 'ANVS', 'HEPA', 'VSLR', 'RUN', 'NVAX', 'AKER', 'AGRX', 'VBIV',
          'VXRT']

In [5]:
today = datetime.date.today()
start_date = '2019-01-01'
end_date = today

In [6]:
def get_data(ticker):
    df = web.DataReader(ticker, data_source = 'yahoo', start = start_date, end = end_date)
    df = df.reset_index()
    df['Symbol'] = ticker
    
    df['Avg_Vol_20'] = -1
    for row in range(20, df.shape[0]):
        df.iloc[row, -1] = np.mean(df.iloc[row-20:row, df.columns.get_loc('Volume')])
    
    df['Vol_Ratio1'] = df['Volume'] / df['Avg_Vol_20']
    
    df['Open/Close'] = -1
    for row in range(1, df.shape[0]):
        df.iloc[row,-1] = df.iloc[row, df.columns.get_loc('Open')] / df.iloc[row-1, df.columns.get_loc('Close')]
    
    df['Low/Open'] = df['Low'] / df['Open']
    df['High/Open'] = df['High'] / df['Open']
    
    df['DailyLogReturn'] = -1
    for row in range(1, df.shape[0]):
        df.iloc[row, -1] = math.log(df.iloc[row, df.columns.get_loc('Close')] / df.iloc[row-1, df.columns.get_loc('Close')])

    df['SquaredDailyLogReturn'] = df['DailyLogReturn']**2
    
    df['SMA10'] = -1
    for row in range(10, df.shape[0]):
        df.iloc[row, -1] = np.average(df.iloc[row-10:row, df.columns.get_loc('Close')])
    
    df['SMA30'] = -1
    for row in range(30, df.shape[0]):
        df.iloc[row, -1] = np.average(df.iloc[row-30:row, df.columns.get_loc('Close')])
        
    df['SMA_Ratio'] = df['SMA30'] / df['SMA10']
    
    Smoothing = 2
    Days = 12
    df['EMA12'] = -1
    #The first observation of EMA will not a have a previous EMA so we will use a 12-day SMA
    df.iloc[12, -1] = (df.iloc[row, df.columns.get_loc('Close')] * Smoothing / (1 + Days)
                       + (np.average(df.iloc[0:12, df.columns.get_loc('Close')]) * (1 - (Smoothing / (1 + Days)))))

    #For the remaining observations, we will use the previous EMA as in the formula
    for row in range(13, df.shape[0]):
        df.iloc[row, -1] = (df.iloc[row, df.columns.get_loc('Close')] * Smoothing / (1 + Days)
                       + (df.iloc[row - 1, -1]) * (1 - (Smoothing / (1 + Days))))    
    
    df['StochasticOscillator'] = -1
    for row in range(13, df.shape[0]):
        df.iloc[row, -1] = ((df.iloc[row, df.columns.get_loc('Close')] - np.min(df.iloc[row-13:row+1, df.columns.get_loc('Low')]))
                            / (np.max(df.iloc[row-13:row+1, df.columns.get_loc('High')]) - np.min(df.iloc[row-13:row+1, df.columns.get_loc('Low')]))
                            * 100)
        
    df = df[(df['Avg_Vol_20'] != -1) & (df['Open/Close'] != -1) & (df['SMA10'] != -1) & (df['SMA30'] != -1)
            & (df['EMA12'] != -1) & (df['StochasticOscillator'] != -1)]
            
    return df 

In [7]:
start_time = time.time()

df = pd.DataFrame()

for ticker in tickers:
    sub_df = get_data(ticker)
    df = pd.concat([df, sub_df])
    
print(time.time() - start_time)

33.989320039749146


In [8]:
df.head(10)

,Date,High,Low,Open,Close,Volume,Adj Close,Symbol,Avg_Vol_20,Vol_Ratio1,Open/Close,Low/Open,High/Open,DailyLogReturn,SquaredDailyLogReturn,SMA10,SMA30,SMA_Ratio,EMA12,StochasticOscillator
30,2019-02-14,23.370001,22.590000,22.740000,23.129999,64441200.0,23.129999,AMD,99802090.0,0.645690,0.995186,0.993404,1.027705,0.012179,0.000148,23.397,21.219667,0.906940,22.992208,66.995071
31,2019-02-15,24.049999,23.200001,23.580000,23.680000,78644100.0,23.680000,AMD,99481685.0,0.790538,1.019455,0.983885,1.019932,0.023500,0.000552,23.269,21.363000,0.918088,23.098022,76.026288
32,2019-02-19,24.410000,23.610001,23.629999,23.950001,57517900.0,23.950001,AMD,99162970.0,0.580034,0.997888,0.999154,1.033009,0.011338,0.000129,23.186,21.584000,0.930907,23.229096,68.435038
33,2019-02-20,24.370001,23.900000,24.139999,23.950001,57091600.0,23.950001,AMD,97632315.0,0.584761,1.007933,0.990058,1.009528,0.000000,0.000000,23.168,21.749000,0.938752,23.340004,58.536618
34,2019-02-21,24.330000,23.850000,24.040001,23.920000,49608200.0,23.920000,AMD,96561210.0,0.513749,1.003758,0.992096,1.012063,-0.001253,0.000002,23.232,21.861667,0.941015,23.429234,64.202327
35,2019-02-22,24.360001,23.879999,24.049999,24.360001,52650700.0,24.360001,AMD,95151055.0,0.553338,1.005435,0.992931,1.012890,0.018228,0.000332,23.298,21.967333,0.942885,23.572429,87.447727
36,2019-02-25,25.520000,24.680000,25.010000,24.709999,63221000.0,24.709999,AMD,92911920.0,0.680440,1.026683,0.986805,1.020392,0.014266,0.000204,23.467,22.106333,0.942018,23.747440,75.076881
37,2019-02-26,24.719999,24.150000,24.650000,24.209999,48470100.0,24.209999,AMD,90560995.0,0.535220,0.997572,0.979716,1.002840,-0.020442,0.000418,23.633,22.272000,0.942411,23.818603,59.692265
38,2019-02-27,24.230000,23.209999,24.110001,23.480000,62649300.0,23.480000,AMD,86226295.0,0.726568,0.995870,0.962671,1.004977,-0.030617,0.000937,23.758,22.403333,0.942981,23.766510,37.230741
39,2019-02-28,23.670000,23.110001,23.209999,23.530001,39384900.0,23.530001,AMD,82798635.0,0.475671,0.988501,0.995692,1.019819,0.002127,0.000005,23.824,22.511667,0.944915,23.730124,38.769238


### Load model

In [9]:
import os
os.chdir('C:\\Users\\Eddie\\Documents\\Stock_Market')

In [10]:
import pickle

In [11]:
filename = 'RandomForest1.sav'

In [12]:
model = pickle.load(open(filename, 'rb'))

In [13]:
input_features = (['Vol_Ratio1',
                 'Open/Close',
                 'Low/Open',
                 'High/Open',
                 'DailyLogReturn',
                 'SquaredDailyLogReturn',
                 'SMA_Ratio',
                 'StochasticOscillator'])

In [14]:
list(set(df['Symbol']))

['VSLR',
 'ANVS',
 'VXRT',
 'VBIV',
 'OGI',
 'AKER',
 'CRVS',
 'HEXO',
 'AGRX',
 'HEPA',
 'RUN',
 'NVAX',
 'SAVE',
 'MOGU',
 'AMD']

### Back testing

In [205]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

#print(color.BOLD + 'Hello World !' + color.END)

#### Investing in 4 stocks at a time at most in one day

In [284]:
def invest(trading_date, holding):
    todays_transactions = pd.DataFrame(columns = ['Purchase_date', 'Sold_date', 'Symbol', 'Price', 'Amount', 'Status',
                                                  'Result', 'Profit'])
    sold = pd.DataFrame(columns = ['Sold_date', 'Symbol', 'Result'])
    
    amount_invested = 0
    amount_returned = 0
    num_invested = 0
    
    
    holding_tickers = list(holding['Symbol'])
    

    for ticker in tickers:
        temp_df = df[(df['Symbol'] == ticker) & (df['Date'] == trading_date)]
        
        if temp_df.shape[0] == 0:
            continue
        
        #check if selling stock
        elif ticker in holding_tickers:
            #need to check if either limit was hit
            low = temp_df.iloc[0, temp_df.columns.get_loc('Low')]
            high = temp_df.iloc[0, temp_df.columns.get_loc('High')]
            purchased_price = holding[holding['Symbol'] == ticker]
            purchased_price = list(purchased_price['Price'])[0]

            
            if low <= purchased_price * stop_loss:
                sold = sold.append({'Sold_date': trading_date,
                                    'Symbol': ticker,
                                    'Result': 'Lost'}, ignore_index = True)
            elif high >= purchased_price * stop_win:
                sold = sold.append({'Sold_date': trading_date,
                                    'Symbol': ticker,
                                    'Result': 'Won'}, ignore_index = True)
        
        #check if buying stock
        else:
            if num_invested == 4:
                continue
                
            inputs = []
            for feature in input_features:
                inputs.append(temp_df.iloc[0, temp_df.columns.get_loc(feature)])
            prediction = model.predict(np.reshape(inputs, (1,8)))

            if prediction == 1: #we buy
                todays_transactions = todays_transactions.append({'Purchase_date': trading_date,
                                                                  'Sold_date': None,
                                                                  'Symbol': ticker,
                                                                  'Price': temp_df.iloc[0, temp_df.columns.get_loc('Close')],
                                                                  'Amount': math.floor(principle / 4),
                                                                  'Status': 'Holding',
                                                                  'Result': None,
                                                                  'Profit': None}, ignore_index = True)
                amount_invested += math.floor(principle / 4)
                num_invested += 1
                #print('Stock %s purchased\n' % ticker)
                #time.sleep(3)
            
    return (amount_invested, todays_transactions, sold)
    


In [314]:
#Update trading_date
def update_trading_date(trading_date):
    return trading_date + datetime.timedelta(days = 1)

In [315]:
stop_win = 1.05
stop_loss = 0.8

In [316]:
#transactions:
transactions = pd.DataFrame(columns = ['Purchase_date', 'Sold_date', 'Symbol', 'Price', 'Amount', 'Status', 'Result',
                                       'Profit'])

In [317]:
principle = 1000
trading_date = datetime.datetime(2019, 2, 14, 0, 0)

In [318]:
i = 1
days_traded = 200
while i <= days_traded:
    if i < 3:
        print(color.BOLD + color.UNDERLINE + 'Date: ' + str(trading_date)[:10] + ' Trading Day: ' + str(i) + color.END + '\n')
    
    holding = transactions[transactions['Status'] == 'Holding']

    #trading occurs for the day
    results = invest(trading_date, holding)

    #principle get adjusted
    amount_invested = results[0]
    principle -= amount_invested
    
    #new transactions added
    transactions = pd.concat([transactions, results[1]], ignore_index = True)
    
    #old transactions updated
    sold_transactions = results[2]
    
    if (sold_transactions.shape[0] == 0) & (amount_invested == 0) & (i < 3):
        print('No stocks purchased or sold today\n')
        time.sleep(3)
    elif (amount_invested != 0) & (i < 3):
        print('%.3f invested today\n' %  amount_invested)
    
    for row in range(sold_transactions.shape[0]):
        sold_ticker = sold_transactions.iloc[row, sold_transactions.columns.get_loc('Symbol')]
        result = sold_transactions.iloc[row, sold_transactions.columns.get_loc('Result')]
        sold_date = sold_transactions.iloc[row, sold_transactions.columns.get_loc('Sold_date')]

        trans_num = transactions[(transactions['Status'] == 'Holding') 
                                 & (transactions['Symbol'] == sold_ticker)].index.values[0]
        
        
        transactions.iloc[trans_num, transactions.columns.get_loc('Sold_date')] = str(sold_date)[:10]
        transactions.iloc[trans_num, transactions.columns.get_loc('Status')] = 'Sold'
        transactions.iloc[trans_num, transactions.columns.get_loc('Result')] = result
    
        amount_invested = transactions.iloc[trans_num, transactions.columns.get_loc('Amount')]
        if result == 'Won':
            transactions.iloc[trans_num, transactions.columns.get_loc('Profit')] = (amount_invested * (stop_win - 1))
            principle += amount_invested * stop_win
        else:
            transactions.iloc[trans_num, transactions.columns.get_loc('Profit')] = (amount_invested * (stop_loss - 1))
            principle += amount_invested * stop_loss
        
        if i < 3:
            print('Stock %s sold, we %s!' % (sold_ticker, result))
            time.sleep(3)
            print('New principle = %.1f' % principle)
            time.sleep(3)
    
    if i == 3:
        print('AND SO ON ...\n')
        print('Give it a few seconds please\n')

    trading_date = update_trading_date(trading_date)
    
    if principle <= 0:
        break
        
    i += 1


account_total = sum(transactions['Amount'].loc[transactions['Status'] == 'Holding']) + principle
print(color.BOLD + color.GREEN + 'ENDING PRINCIPLE AFTER %s days = %.3f' % (str(days_traded), account_total) + color.END)

Date: 2019-02-14 Trading Day: 1

No stocks purchased or sold today

Date: 2019-02-15 Trading Day: 2

750.000 invested today

AND SO ON ...

Give it a few seconds please

ENDING PRINCIPLE AFTER 200 days = 1037.450
